In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents import AgentExecutor
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain import hub
from langchain.tools.render import render_text_description_and_args
from langchain.output_parsers import PydanticOutputParser
from langchain.pydantic_v1 import BaseModel, Field

from utils import map_openbb_collection_to_langchain_tools

In [3]:
# Let's set-up our tool retrieval

fundamental_openbb_tools = map_openbb_collection_to_langchain_tools(
    "/equity/fundamental"
)

docs = [
    Document(page_content=t.description, metadata={"index": i})
    for i, t in enumerate(fundamental_openbb_tools)
]

vector_store = FAISS.from_documents(docs, OpenAIEmbeddings())

In [4]:
retriever = vector_store.as_retriever(search_kwargs={"k": 2})  # <- now returns top 2


def get_tools(query):
    docs = retriever.get_relevant_documents(query)
    return [fundamental_openbb_tools[d.metadata["index"]] for d in docs]


# Quick test
fetched_tools = get_tools("income statement")
len(fetched_tools)

2

In [5]:
# Let's make it easy to create react agents, since we'll need a lot of them later.
def langchain_react_agent(tools):
    """Define a ReAct agent bound with specific tools."""
    prompt = hub.pull("hwchase17/react-multi-input-json")
    prompt = prompt.partial(
        tools=render_text_description_and_args(tools),
        tool_names=", ".join([t.name for t in tools]),
    )

    llm = ChatOpenAI(model="gpt-4-1106-preview").bind(stop=["\nObservation"])

    chain = (
        {
            "input": lambda x: x["input"],
            "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
        }
        | prompt
        | llm
        | JSONAgentOutputParser()
    )

    agent_executor = AgentExecutor(
        agent=chain,
        tools=tools,
        verbose=False,
        return_intermediate_steps=False,
        handle_parsing_errors=True,
    )
    return agent_executor


agent_executor = langchain_react_agent(tools=fetched_tools)
result = agent_executor.invoke({"input": input})

In [9]:
# The primary goal is to
# 1. Break a larger question down into subquestions + the appropriate query to fetch the right tool to answer the subquestion
# 2. Retrieve the right tools for each subquestion
# 3. Answer each subquestion using a ReAct agent
# 4. To combine all of the subquestion answers to generate a final answer.

# Part 1 break it into subquestions


# We'll use Pydantic to do some output enforcement
# (It's just prompting and parsing under the hood)
class SubQuestion(BaseModel):
    question: str = Field(description="The subquestion itself.")
    query: str = Field(
        description="The query to pass to the `fetch_tools` function to retrieve the appropriate tool to answer the question."
    )


class SubQuestionList(BaseModel):
    subquestions: list[SubQuestion] = Field(
        description="The list of SubQuestion objects."
    )


subquestion_parser = PydanticOutputParser(pydantic_object=SubQuestionList)

system_message = """\
You are a world class state of the art agent.

You have access to multiple tools, via a "fetch_tools" function that will retrieve the tools you need.
The `fetch_tools` function accepts a string of keywords as input specifying the type of tool to retrieve.
Each retrieved tool represents a different data source or API that can retrieve the required data.

Your purpose is to help answer a complex user question by generating a list of sub
questions, as well as the corresponding keyword query to the "fetch_tools" function
to retrieve the relevant tools to answer each corresponding subquestion.

These are the guidelines you consider when completing your task:
* Be as specific as possible
* Avoid using acronyms
* The sub questions should be relevant to the user question
* The sub questions should be answerable by the tools retrieved by the query to `fetch_tools`
* You can generate multiple sub questions
* You don't need to query for a tool if you don't think it's relevant

## Output format
{format_instructions}

### Example responses
```json
{{"subquestions": [
    {{
        "question": "What are the latest financial statements of AMZN?", 
        "query": "financial statements"
    }}, 
    {{
        "question": "What is the most recent revenue and profit margin of AMZN?", 
        "query": "revenue profit margin ratios"
    }}, 
    {{
        "question": "What is the current price to earnings (P/E) ratio of AMZN?", 
        "query": "ratio price to earnings"
    }}, 
    {{
        "question": "What is the current debt to equity ratio of AMZN?", 
        "query": "debt equity ratio"
    }}
]}}
```

"""

human_message = """\
    ## User Question
    {input}
    """

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
        ("human", human_message),
    ]
)
prompt = prompt.partial(format_instructions=subquestion_parser.get_format_instructions())

llm = ChatOpenAI(
    model="gpt-4"
)  # gpt-3.5-turbo works well, but gpt-4-1106-preview isn't good at returning JSON.

subquestion_chain = {"input": lambda x: x["input"]} | prompt | llm | subquestion_parser

# Our high-level question we're going to attempt to answer
INPUT = "Perform a fundamentals financial analysis of AMZN using the most recently available data. What do you find that's interesting?"

subquestion_list = subquestion_chain.invoke({"input": INPUT})

for subquestion in subquestion_list.subquestions:
    print(subquestion)  # We probably won't have all the right tools loaded to answer these questions. Improvements for the future!

question='What are the latest financial statements of AMZN?' query='financial statements'
question='What is the most recent revenue and profit margin of AMZN?' query='revenue profit margin ratios'
question='What is the current price to earnings (P/E) ratio of AMZN?' query='ratio price to earnings'
question='What is the current debt to equity ratio of AMZN?' query='debt equity ratio'
question="What are AMZN's key financial ratios and how do they compare to industry averages?" query='financial ratios comparison'
question="What are the recent trends in AMZN's sales and earnings growth?" query='sales earnings growth trends'
question='What is the current market capitalization of AMZN and how has it changed over the past year?' query='market capitalization history'
question='What are the recent significant financial events or news related to AMZN?' query='financial news events'


In [10]:
# Part 2 is to fetch the appropriate tool for each subquestion
# (We'll create a new data structure here we can re-use)

subquestions_and_tools = []

for subquestion in subquestion_list.subquestions:
    tools = get_tools(subquestion.query)
    subquestions_and_tools.append(
        {
            "subquestion": subquestion.question,
            "query": subquestion.query,
            "tools": tools,
        }
    )

len(subquestions_and_tools)

8

In [11]:
# Part 3 is to answer each of the subqueries. We'll use a ReAct agent paired with the subquestion and appropriate tools to do this.
from langchain.schema.runnable import RunnableParallel

agents = []
for i, subquestion in enumerate(subquestions_and_tools):
    react_agent = langchain_react_agent(tools=subquestion["tools"])
    agents.append(react_agent)

len(agents)

8

In [13]:
# Run the agents to answer the subquestions
for i, subquestion in enumerate(subquestions_and_tools):
    input = f"""\
    Given the following high-level question: {INPUT}
    Answer only the following subquestion: {subquestion}

    Give your answer in a bullet-point list.
    Explain your reasoning, and make reference to the retrieved data.

    Example output format:
    ```
    - <the first observation, insight, and/or conclusion> 
    - <the second observation, insight, and/or conclusion> 
    - <the third observation, insight, and/or conclusion> 
    ... REPEAT AS MANY TIMES AS NECESSARY TO ANSWER THE SUBQUESTION.
    ```

    Return only your answer as a bulleted list as a single string. Don't respond with JSON or any other kind of datastructure.
    """
    try:
        result = agents[i].invoke({"input": input})
        output = result["output"]
    except Exception:  # Terrible practice, but it'll do for now.
        output = "I was unable to answer the subquestion using the available tool."  # We'll include the error message in the future

    print(subquestion["subquestion"])
    print("----")
    print(output)
    print("=======")

    # We'll misbehave and re-use the same datastructure again
    subquestion["observation"] = output

What are the latest financial statements of AMZN?
----
- AMZN reported a significant revenue of $513.98 billion for the fiscal year ending December 31, 2022, which indicates the vast scale of its operations.
- The company's cost of revenue was $288.83 billion, resulting in a gross profit of $225.15 billion, which shows a gross profit ratio of approximately 43.8%.
- Research and development expenses were notably high at $73.21 billion, reflecting Amazon's continued investment in innovation.
- AMZN had a challenging year with a net loss of $2.72 billion, an unexpected turn given the company's scale and market position.
- The balance sheet shows a strong cash position with $53.89 billion in cash and cash equivalents, which is critical for the company's liquidity.
- Total assets were reported at $462.68 billion, and the company has a substantial long-term debt of $140.12 billion, indicating a leverage ratio that stakeholders should monitor.
- The net debt position is $86.23 billion, consid

In [14]:
# Part 4 is to generate a response based on the answers to each of the subquestions

def render_subquestions_and_answers(subquestions):
    output = ""
    for subquestion in subquestions:
        output += "Subquestion: " + subquestion["subquestion"] + "\n"
        output += "Observations: \n" + subquestion["observation"] + "\n\n"

    return output

system_message = """\
    Given the following high-level question: 

    Question: {input}

    And the following subquestions and subsequent observations:

    {subquestions}

    Answer the high-level question. Give your answer in a bulleted list.
    """


prompt = ChatPromptTemplate.from_messages([("system", system_message)])

llm = ChatOpenAI(model="gpt-4")  # Let's use the big model for the final answer.

final_chain = (
    {
        "input": lambda x: x["input"],
        "subquestions": lambda x: render_subquestions_and_answers(x["subquestions"]),
    }
    | prompt
    | llm
)

result = final_chain.invoke({"input": INPUT, "subquestions": subquestions_and_tools})

In [15]:
print(result.content)  # Et voila

- Amazon's latest financial statements show a massive scale of operations, with revenue of $513.98 billion for the fiscal year ending December 31, 2022. Despite the significant revenue, the company reported a net loss of $2.72 billion.
- The company has a strong cash position with $53.89 billion in cash and cash equivalents, which ensures liquidity. However, the company also has a substantial long-term debt of $140.12 billion, leading to a net debt position of $86.23 billion.
- The gross profit margin for the fiscal year 2022 was approximately 43.81%, but the net profit margin was negative at -0.53%, showing that the company incurred more expenses than revenue.
- Amazon's current P/E ratio is 75.04, which is quite high. The high P/E ratio may suggest that investors expect high earnings growth in the future or that the stock is overvalued.
- The company's current debt to equity ratio is approximately 0.96, indicating a balanced but significant reliance on debt financing relative to its 